In [1]:
from dataclasses import dataclass
from itertools import combinations
from typing import Callable, Any, List, Tuple

import pandas as pd
from pandas import DataFrame, Series, NamedAgg

In [2]:
df = pd.read_excel("test_data.xlsx")
print(f"Imported {df.shape[0]:,} rows x {df.shape[1]:,} columns")

Imported 12,736 rows x 16 columns


In [13]:
@dataclass
class NamedTotal:
    __slots__ = ['column', 'alias', 'selector']
    
    column: str
    alias: str
    selector: Callable[[Any], bool]

Subset = Tuple[NamedTotal]

def _generate_valid_subsets(totals: List[NamedTotal], required: List[str] = None) -> Subset:
    """ Yield a tuple of all NamedTotals that comprise a valid combination.

    Args
    ----
    totals: list of NamedTotal instances that define the subsets.
    required: list of column names that will only output aggregations that include total/subtotal selections.

    Yields
    -------
    Tuple of NamedTotal instances for a single aggregation.
    """
    def validator(subset) -> bool:
        """ Return if the given subset contains no duplicated cols and all required cols """
        columns_in_subset = [c for c, _, _ in subset]
        is_distinct = len(columns_in_subset) == len(set(columns_in_subset))
        has_required = True if required is None else set(required).issubset(columns_in_subset)
        return is_distinct & has_required

    subsets = []
    for combin in [combinations(totals, r) for r in range(1, len(totals) + 1)]:
        for subset in combin:
            subsets.append(subset)    
    for valid in filter(validator, subsets):
        yield valid


def _create_subset_frame(df: pd.DataFrame, subset: Subset) -> pd.DataFrame:
    """ Apply the subset definition to filter and transform the DataFrame
    
    Args
    ----
    df: DataFrame to be filtered and transformed
    subset: Definition of the column filters and alias values

    Returns
    -------
    Selected rows from df as defined by the selector functions in subset
    
    """
    for col, alias, selector in subset:
        df=df.loc[df[col].map(selector)].assign(**{col: alias})
    return df

In [ ]:
def aggregator(grp: DataFrame, measures: dict) -> Series:
    d = {}
    for alias, func in measures.items():
        d[alias] = func(grp)
    return Series(d, index=d.keys())

In [ ]:
def dataframe_combination_agg(df: DataFrame, groupby: List[str], totals: List[NamedTotal], aggregator: Callable[[DataFrame], Series], totals_only: List[str], csv_output_path: str = None) -> DataFrame:
    """

    Args
    ----
    df: DataFrame to be aggregated
    groupby: Column names to groupby
    totals: List of NamedTotals which defines the total/subtotals in the combination aggs
    aggregator: Callable that transforms a single grouped dataframe into a series of measures.
    totals_only: List of column names that will only include their alias values defined in totals in the combination
    output.
    csv_output_path: Optional path to a csv file to output each aggregation instead of storing in memory. Useful when
    there are many combinations being created on a large dataframe.

    Returns
    -------
    Original dataframe when output to csv, otherwise the result of applying all aggregations.
    """
    for subset in _generate_valid_subsets(totals, totals_only):
        agg = _create_subset_frame(df, subset).apply(aggregator)      
        

        # TODO | split this into append to list for pd.concat return
        # or csv.append
        if csv_output_path is None:
            aggregations = []
        
        else:
            with open(csv_output_path, 'a') as csv:
                pass


In [18]:
totals = [
    ('Type', 'Grain Carriers', lambda v: v in ['Bulk Carrier', 'General Cargo']),
    ('Actual Draft', 'FSD Capable', lambda v: v <= 10),
    ('Actual Draft', 'FSD Incapable', lambda v: v > 10),
]
valid_subsets = list(_generate_valid_subsets(totals))

In [21]:
test_frame = _create_subset_frame(df, valid_subsets[0])

In [22]:
test_frame[['Actual Draft', 'Type']].value_counts()

Actual Draft  Type          
7.00          Grain Carriers    121
7.50          Grain Carriers     86
7.20          Grain Carriers     83
7.30          Grain Carriers     75
6.60          Grain Carriers     73
                               ... 
14.54         Grain Carriers      1
14.57         Grain Carriers      1
14.72         Grain Carriers      1
14.75         Grain Carriers      1
18.70         Grain Carriers      1
Length: 1035, dtype: int64

In [ ]:
for subset in valid_subsets:
    print(subset[['Actual Draft', 'Type']].value_counts())

In [ ]:
for s in valid_subsets:
    print(s)